# Hibridación de datos obtenidos mediante contadores inteligente

## Carga de estaciones
Cargamos los datos de las estaciones, con geoposicionamiento en UTM. Estas estaciones las hemos obtenido haciendo uso de la API de AEMET, vease [ejemplo AEMET](https://github.com/itelligent-mrivas/CU_contadores_inteligentes/blob/main/AEMET.ipynb)

In [1]:
import pandas as pd

__path_estaciones = './datasets/estaciones.csv'

dfEstaciones = pd.read_csv(__path_estaciones, sep=',')
dfEstaciones.head(10)

,indicativo,nombre,provincia,altitud,Lat,Long,x_UTM,y_UTM
0,3100B,ARANJUEZ,MADRID,540,40.0402,-3.33,471847.797443,4.432271e+06
1,3110C,BUITRAGO DEL LOZOYA,MADRID,1026,41.0025,-3.33,472247.521766,4.539087e+06
2,3191E,COLMENAR VIEJO,MADRID,1004,40.4146,-3.43,463518.224035,4.473864e+06
3,3200,GETAFE,MADRID,620,40.1758,-3.43,463389.571829,4.447358e+06
4,3129,MADRID AEROPUERTO,MADRID,609,40.2800,-3.33,471946.664732,4.458887e+06
5,3194U,"MADRID, CIUDAD UNIVERSITARIA",MADRID,664,40.2706,-3.43,463440.568481,4.457881e+06
6,3196,"MADRID, CUATRO VIENTOS",MADRID,690,40.2232,-3.43,463415.057580,4.452619e+06
7,3195,"MADRID, RETIRO",MADRID,667,40.2443,-3.43,463426.410593,4.454961e+06
8,3266A,PUERTO ALTO DEL LE�N,MADRID,1532,40.4223,-4.04,411774.417837,4.475149e+06
9,2462,PUERTO DE NAVACERRADA,MADRID,1894,40.4735,-4.04,411841.350820,4.480833e+06


## Carga de los datos de los usuarios
Cargamos los datos de los usuarios que vamos a procesar.

**NOTA** Los datos han sido anonimizados añadiendo ruido a las coordenadas, por lo que cualquier coincidencia se trataría de una mera casualidad.

In [2]:
__path_usuarios = './datasets/usuarios.csv'

dfUsuarios = pd.read_csv(__path_usuarios, sep=',')
dfUsuarios.head(10)

,altitud,utmX,utmY
0,591.632996,469895.073415,4.483238e+06
1,693.760071,445803.172256,4.488764e+06
2,653.997803,447036.012700,4.488402e+06
3,692.813904,429389.184144,4.466935e+06
4,681.574463,429316.612907,4.466035e+06
5,690.266174,430896.667997,4.466668e+06
6,685.127380,432447.440354,4.467681e+06
7,695.080933,429447.195166,4.467305e+06
8,690.918579,424192.298867,4.473986e+06
9,641.393799,468429.021557,4.489461e+06


## Función para el cálculo de la distancia
Usaremso la distancia Euclidea para medir la distancia entre dos puntos. Al trabajar con cooredenas UTM esta distancia se corresponde a metros.

In [3]:
import math

def distancia(X1, Y1, X2, Y2):
    Xd = X2 - X1
    Yd = Y2 - Y1
    
    return math.sqrt(Xd*Xd + Yd*Yd)

## Función para la corrección de la temperatura por altitud

Se corige 0.6 grados por cada 500m de diferencia

In [4]:
def correcionTemperatura(aUsuario, aEstacion):
    dif = aEstacion - aUsuario
    
    correccion = (abs(dif)//100)*0.6
    if(dif < 0):
        correccion = correccion*-1
        
    return correccion

## Cálculo de la distancia usuario-estación
Para cada usuario calculamos cual es la estación más cercana. Una vez tenemos esta, calculamos el factor de correción para la temperatura segun la diferencia de altitud.

In [8]:
import heapq

#Path donde se almacenara el csv de resultados
__path_salida = './salida/UsuarioEstaciones.csv'

#Creamos el dataFrame en el que se alamcenaran los resultados
dfResultado = pd.DataFrame(columns=['idUsuario','Indicativo','Distancia','FactorCorrecion'])

for indexUsuario, usuario in dfUsuarios.iterrows():
    heap = []
    
    #Se calcula la distancia del usuario a todas las estaciones, manteniendo un min heap para eficiencia
    for indexEstacion, estacion in dfEstaciones.iterrows():
        d = distancia(usuario.utmX, usuario.utmY, estacion.x_UTM, estacion.y_UTM)
        heapq.heappush(heap, (d, estacion))
           
    #Selecciono la estación mas cercana y se almacena 
    elto = heapq.heappop(heap)
    factorCorreccion = correcionTemperatura(usuario.altitud, elto[1].altitud)
    
    #Creamos la fila con el usuario, su estación mas cercana y el factor de correción
    resultado = {
        'idUsuario':indexUsuario, 
        'Indicativo': elto[1].indicativo, 
        'Distancia':elto[0], 
        'FactorCorrecion':factorCorreccion}
    
    #append row to the dataframe
    dfResultado= dfResultado.append(resultado, ignore_index=True)

dfResultado.to_csv(__path_salida, index = False)
dfResultado.head(10)


,idUsuario,Indicativo,Distancia,FactorCorrecion
0,0,3191E,11337.114316,2.4
1,1,3191E,23148.151284,1.8
2,2,3191E,21977.544128,1.8
3,3,3266A,19435.874002,4.8
4,4,3266A,19768.483430,4.8
5,5,3266A,20918.611642,4.8
6,6,3266A,21980.730755,4.8
7,7,3266A,19335.620857,4.8
8,8,3266A,12472.243037,4.8
9,9,3191E,16351.355970,1.8
